In [1]:
from instagram import Account, Media, WebAgent #взять отсюда https://github.com/OlegYurchik/pyInstagram
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms

In [3]:
MAX_COUNT = 20 #сколько максимальо фоток мы хотим загрузить
accountName = "mama_na_kuxne" #название аккаунта

agent = WebAgent()
account = Account(accountName)

link_list = []
pointer = None

media, pointer = agent.get_media(account, count = MAX_COUNT) 
for med in media:
    try:
        link_list.append(med.resources[0])
    except Exception:
        continue

In [4]:
img_dimensions = 224

img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

resnet_loaded = torch.hub.load('pytorch/vision', 'resnet18')
resnet_loaded.fc = nn.Sequential(nn.Linear(resnet_loaded.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, 2))
resnet_loaded.load_state_dict(torch.load('./models/resnet18.pth', map_location=torch.device('cpu')))
resnet_loaded.eval()

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

Downloading: "https://github.com/pytorch/vision/archive/master.zip" to /Users/yury/.cache/torch/hub/master.zip


In [11]:
import requests
from PIL import Image
from io import BytesIO

img_list = [] #здесь будут храниться PIL-image фотографий еды
food_link_list = []
for link in link_list:
    try:
        response = requests.get(link, stream=True)
        img_pic = Image.open(BytesIO(response.content))
        img = img_transforms(img)
        img = img.unsqueeze(0)
        resnet_loaded.to(device)
        prediction = resnet_loaded(img.to(device))
        prediction = prediction.argmax()
        if (prediction == 1):
            img_list.append(img)
            food_link_list.append(link)
            print(link)
        del response
    except Exception:
        continue



https://instagram.fhrk1-1.fna.fbcdn.net/v/t51.2885-15/e35/c0.180.1440.1440a/s150x150/126336206_865236640682978_2344174815437822524_n.jpg?_nc_ht=instagram.fhrk1-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=mN7Mo44F0coAX-9mGE7&tp=1&oh=0bd85eab4d3f02c10d259ee93754758e&oe=5FE35293
https://instagram.fhrk1-1.fna.fbcdn.net/v/t51.2885-15/e35/s150x150/123228945_3329629723802191_5889572144787479607_n.jpg?_nc_ht=instagram.fhrk1-1.fna.fbcdn.net&_nc_cat=107&_nc_ohc=XDrzhQWjNS4AX-QJoIk&tp=1&oh=1e3f2e34879915188f2b7717fd4ee524&oe=5FBC718E
https://instagram.fhrk1-1.fna.fbcdn.net/v/t51.2885-15/e35/c0.180.1440.1440a/s150x150/122828130_1004110713391430_4407864791860020883_n.jpg?_nc_ht=instagram.fhrk1-1.fna.fbcdn.net&_nc_cat=102&_nc_ohc=DDVY1QuNbQwAX-d38oX&tp=1&oh=cb53a26625707ba0ae2a0a36e89424f1&oe=5FE27FE0
https://instagram.fhrk1-1.fna.fbcdn.net/v/t51.2885-15/e35/p150x150/122724098_682436929051109_9081446399525587438_n.jpg?_nc_ht=instagram.fhrk1-1.fna.fbcdn.net&_nc_cat=105&_nc_ohc=3dOtRO-Y2MIAX_jXkXw&tp=1&oh=99911